In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.6.0


## IMDB 데이터셋 다운로드

In [2]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)
# num_words = 10000 : 훈련데이터에서 가장 많이 등장하는 상위 10000개의 단어 선택

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print("훈련 샘플 : {}, 레이블 : {}".format(len(train_data), len(test_labels)))

훈련 샘플 : 25000, 레이블 : 25000


In [4]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
len(train_data[0]), len(train_data[1]) #영화 리뷰들은 길이가 다름

(218, 189)

In [9]:
word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

reversed_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reversed_word_index.get(i, '?') for i in text])

In [10]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [13]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [14]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [15]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [16]:
vocab_size = 10000

In [18]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape = (None,)))
# 정수로 인코딩된 단어를 입력 받아 각 단어 인덱스에 해당하는 임베팅 벡터를 찾음
# 최종 차원 : (batch, sequence, embedding)
model.add(keras.layers.GlobalAveragePooling1D())
# sequence 차원에 대해 평균을 계산하여 각 샘플에 대해 고정된 길이의 출력 벡터를 반환
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))
#출력 노드를 가진 완전 연결 층

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [21]:
history = model.fit(partial_x_train, partial_y_train, epochs = 40,
                    batch_size = 512, validation_data = (x_val, y_val), verbose = 1)

Epoch 1/40
30/30 [==============================] - 3s 16ms/step - loss: 0.6923 - accuracy: 0.5495 - val_loss: 0.6912 - val_accuracy: 0.5133
Epoch 2/40
30/30 [==============================] - 0s 10ms/step - loss: 0.6882 - accuracy: 0.6351 - val_loss: 0.6850 - val_accuracy: 0.6639
Epoch 3/40
30/30 [==============================] - 0s 9ms/step - loss: 0.6780 - accuracy: 0.7280 - val_loss: 0.6714 - val_accuracy: 0.7322
Epoch 4/40
30/30 [==============================] - 0s 10ms/step - loss: 0.6580 - accuracy: 0.7553 - val_loss: 0.6480 - val_accuracy: 0.7578
Epoch 5/40
30/30 [==============================] - 0s 10ms/step - loss: 0.6269 - accuracy: 0.7828 - val_loss: 0.6153 - val_accuracy: 0.7719
Epoch 6/40
30/30 [==============================] - 0s 10ms/step - loss: 0.5864 - accuracy: 0.8019 - val_loss: 0.5739 - val_accuracy: 0.7988
Epoch 7/40
30/30 [==============================] - 0s 10ms/step - loss: 0.5383 - accuracy: 0.8252 - val_loss: 0.5289 - val_accuracy: 0.8153
Epoch 8/40
30/

In [23]:
results = model.evaluate(test_data, test_labels, verbose=2)
print(results)
# loss, accuracy

782/782 - 1s - loss: 0.3289 - accuracy: 0.8721
[0.32889044284820557, 0.8721200227737427]


In [24]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [25]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label = 'Training loss') # 'bo' : 파란색 점
plt.plot(epochs, val_loss, 'b', label = 'Validation loss') # 'b' : 파란색 선
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.legend()

plt.show()

In [26]:
plt.clf()

plt.plot(epochs, acc ,'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()